In [1]:
%load_ext autoreload
%autoreload 2
from train import run
import os
import os.path as osp
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

In [ ]:
with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")
    print(cfg)
    data_root = cfg.data.data_root
    if not osp.exists(data_root):
        os.makedirs(data_root, exist_ok=True)
        data_root_father = osp.dirname(data_root)
        !wget https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip -O {data_root_father}/data.zip
        !unzip {data_root_father}/data.zip -d {data_root}
    # run(cfg)

In [ ]:
# wandb
import wandb
wandb.login()


# Define a sweep

In [2]:
sweep_configuration = {
    "name": "sweep-hyperparams",
    "method": "bayes",
    "metric": {"goal": "maximize", "name": "epoch_val_dice_score"},
    "parameters": {
        "learning_rate": {
            "values": [1e-3, 1e-4, 1e-5],

        },
        "batch_size": {"values": [16, 32, 64]},
        "epochs": {"values": [30, 120, 300]},
        "img_size": {
            "values": [64, 128, 256],
        },
    },
}

# sweep_id = "b1p8nwvg"
sweep_id = wandb.sweep(sweep=sweep_configuration, project="sweep-hyperparams")


# Sweep run

In [9]:
from train import run
def sweep_train():
    from hydra import initialize, compose
    wandb.init()
    with initialize(version_base=None, config_path="conf"):
        cfg = compose(config_name="config",
                      overrides=[f"train.optimizer.lr={wandb.config.learning_rate}",
                                 f"train.batch_size={wandb.config.batch_size}",
                                 f"train.num_epochs={wandb.config.epochs}",
                                 f"train.img_size={wandb.config.img_size}"])
        
        run(cfg)

In [ ]:
# sweep_id = wandb.sweep(sweep=sweep_configuration, project="pedestrian-detection")
wandb.agent(sweep_id, function=sweep_train)